<a href="https://colab.research.google.com/github/GunduSriBhanu/LLM_Vector_Stores/blob/main/Langchain_Opensearch_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check the List of libraries dependence presence

In [ ]:
pip list | grep detectron2


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Installing all the libraries

In [ ]:
!pip install --upgrade langchain openai  -q
!pip install sentence_transformers -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install poppler-utils
!pip install tiktoken -q
!pip install pypdf
!pip install requests-aws4auth
!pip install requests --upgrade
!pip install opensearch-py
!pip install aws-requests-auth


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


# Importing all Libraries

In [ ]:
import langchain
from langchain.document_loaders import DirectoryLoader
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.autonotebook import tqdm
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFDirectoryLoader
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import re
import requests
import json
import pandas as pd
import boto3
from requests_aws4auth import AWS4Auth
from langchain.vectorstores import OpenSearchVectorSearch
from opensearchpy import OpenSearch
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

/tmp/ipykernel_364/3577097376.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Authentication Keys for performing LLM

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = ""
os_host = 'search-demo-opensearch-yyeyn2fj4nqty5gxo65ozakwxy.us-west-2.es.amazonaws.com'#'search-demo-opensearch-yyeyn2fj4nqty5gxo65ozakwxy.us-west-2.es.amazonaws.com'
os_port = 443
os_auth = ('admin_sri', '')
os_url = 'search-demo-opensearch-yyeyn2fj4nqty5gxo65ozakwxy.us-west-2.es.amazonaws.com/_dashboards'
aws_access_key_id = ''
aws_secret_access_key = ''
aws_region = 'us-west-2'
opensearch_endpoint_url = 'https://search-demo-opensearch-yyeyn2fj4nqty5gxo65ozakwxy.us-west-2.es.amazonaws.com'

# Step 1: Loading pdf documents for training

In [ ]:
loader = PyPDFDirectoryLoader("data-set/")
documents = loader.load()
documents[0]

# Text Splitting in a documents

In [ ]:
def split_docs(documents,chunk_size=2000,chunk_overlap=200):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))
docs[0].page_content

# Embeddings

In [ ]:
embeddings = OpenAIEmbeddings(model_name="ada")

query_result = embeddings.embed_query("Hello world")
DERIVED_EMB_SIZE = len(query_result)
DERIVED_EMB_SIZE

1536

# Checking the encoding and resolving the encoding

In [ ]:
def clean_text(text):
    # Remove newline characters ("\n")
    cleaned_text = text.replace("\n", " ")

    # Remove Unicode escape sequences ("\uXXXX")
    cleaned_text = re.sub(r'\\u[0-9a-fA-F]+', '', cleaned_text)

    return cleaned_text


input_data = []
for i in range(len(documents)):
   input_data.append(clean_text(documents[i].page_content))

# Print the cleaned text
print(input_data[0])

# Opensearch Operations:
## Check the opensearch connectivity

In [ ]:
os_client = OpenSearch(
    hosts = [{'host':os_host, 'port':443}],
    http_auth = os_auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 30
)
os_client.info()

{'name': '0b18e22b0bb1b8ccd78b596109d5da5a',
 'cluster_name': '054719795948:demo-opensearch',
 'cluster_uuid': 'vprlqmRwS2KrsCnXmZBjKg',
 'version': {'distribution': 'opensearch',
  'number': '2.9.0',
  'build_type': 'tar',
  'build_hash': 'unknown',
  'build_date': '2023-09-18T10:22:23.747603002Z',
  'build_snapshot': False,
  'lucene_version': '9.7.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

## Delete if already index exists

In [ ]:
if(False):
    index_name = 'vs-opensearch'
    response = os_client.indices.delete(index_name)
    response
#os_client.indices.delete('vs-opensearch')

## Creating index in opensearch

In [ ]:
index_name = "vs-opensearch"
index_body = {
    "settings": {
        "index": {
            "number_of_shards": 5,

            "knn": True,
            "knn.algo_param.ef_search": 512
        },
    },
    "mappings": {
        "dynamic_templates": [
            {
                # map all metadata properties to be keyword
                "metadata.*": {
                    "match_mapping_type": "*",
                    "mapping": { "type": "keyword" },
                },
            },
        ],
        "properties": {
            "text": { "type": "text" },
            "metadata": { "type": "object" },
            "vector_field": { "type": "knn_vector", "dimension": DERIVED_EMB_SIZE },
            "embedding": {
                "type": "knn_vector",
                "dimension": DERIVED_EMB_SIZE,
                "method": {
                    "name": "hnsw",
                    "engine": "nmslib",
                    "space_type": "l2",
                    "parameters": { "ef_construction": 512, "m": 16 }
                }
            }
        }
    }
}


response = os_client.indices.create(index_name, body=index_body)
response

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'vs-opensearch'}

## Searching from OpenSearch Documents

In [ ]:
service = 'es' # must set the service as 'es'
region = 'us-west-2'
credentials = boto3.Session(aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key).get_credentials()
#awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)
awsauth = AWS4Auth('admin_sri', '', region,service, session_token=credentials.token)

docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    embeddings,
    opensearch_url=[{'host':os_host, 'port':443}],
    http_auth=os_auth, #awsauth,
    timeout = 300,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name="vs-opensearch",
)

In [ ]:
docsearch

# Training the LLM Model

In [ ]:
model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name)

from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### Getting answer for the query

In [ ]:
def get_answer(query):
  similar_docs = docsearch.similarity_search(query,k=2)#get_similiar_docs(query)
  #print(similar_docs)
  answer =  chain.run(input_documents=similar_docs, question=query)
  return  answer

In [ ]:
query = "what are all the sections in document?"
get_answer(query)

In [ ]:
query = "what are the sections in Block 28-22 docuemnt?"
get_answer(query)